In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io

In [12]:
# The dataset includes the gene expression values of five
# types of cancer: lung adenocarcinoma (LUAD),
# breast invasive carcinoma (BRCA), kidney renal clear cell
# carcinoma (KIRC), lung squamous cell carcinoma (LUSC),
# and uterine corpus endometrial carcinoma (UCEC).

# https://data.mendeley.com/datasets/sf5n64hydt/1
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GPL20573

In [7]:
dir = os.path.join('sf5n64hydt-1', 'cancer types.mat')
mat = scipy.io.loadmat(dir)
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Aug 01 17:46:29 2017',
 '__version__': '1.0',
 '__globals__': [],
 'cancerTypes': array([[array(['BRCA'], dtype='<U4')],
        [array(['BRCA'], dtype='<U4')],
        [array(['BRCA'], dtype='<U4')],
        ...,
        [array(['UCEC'], dtype='<U4')],
        [array(['UCEC'], dtype='<U4')],
        [array(['UCEC'], dtype='<U4')]], dtype=object),
 'data': array([[44.02354236,  9.21628619, 11.31907844, ...,  4.2956195 ,
          8.76876844,  1.        ],
        [29.74615746,  9.7656002 , 40.540128  , ...,  3.81901941,
          5.75850117,  1.        ],
        [35.79931536,  9.88478062,  3.886043  , ...,  4.38238737,
          5.30617722,  1.        ],
        ...,
        [14.66541424, 20.19564581,  6.7034771 , ...,  1.83021579,
          2.35529186,  5.        ],
        [31.11602192, 14.56206615,  9.12158501, ...,  1.17613142,
          1.89129728,  5.        ],
        [74.04662769, 13.00396207,  9.47227364